### Assignment 2: EDA and Prediction with Logistic Regression 

**You must push a version before next Tuesday at class time!** 

The Heart Disease Prediction dataset provides vital insight in the relationship between risk factors and cardiac health. This dataset contains 270 case studies of individuals classified as either having or not having heart disease based on results from cardiac catheterizations - the gold standard in heart health assessment. Each patient is identified by 13 independent predictive variables:

age (Age of the patient in years)
sex (Female = 0/Male = 1)
cp chest pain type ([typical angina, atypical angina, non-anginal, asymptomatic])
trestbps resting blood pressure (resting blood pressure (in mm Hg on admission to the hospital))
chol (serum cholesterol in mg/dl)
fbs (if fasting blood sugar > 120 mg/dl, yes = 1, no = 0)
restecg (resting electrocardiographic results)
-- Values: [normal = 0, stt abnormality = 1, lv hypertrophy = 2]
MaxHR: maximum heart rate achieved in stress test
exang: exercise-induced angina (True = 1/ False 0)
oldpeak: ST depression induced by exercise relative to rest
slope: the slope of the peak exercise ST segment
ca: number of major vessels (0-3) colored by fluoroscopy
thal: [normal; fixed defect; reversible defect]
AHD: The confirmed heart disease based on cardiac catherizations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler  #this is new!


In [ ]:
heart = pd.read_csv('Heart.csv')

The very first thing to check in any data file, is whether there are any missing values. The pandas method isna will inform you if there is any missing data.  

In [ ]:
heart.isna().sum()


The table above shows you the number of missing values that you will find in the data file.  In this case, I gave you a file with missing values.  This will always occcur in real world data.  

The `.dropna()` method can be used to remove any data with missing values.  

In [ ]:
heart = heart.dropna()

2. Use pandas method to summarize the properties of the data. Use `head` and `info`. At the end of this set of code are some questions for you to answer.  

In [ ]:
heart.head()

In [ ]:
heart.info()

Identify which variables are categorical, and which variables are numeric.

Answer: 

Are there any missing data? 

Answer: 

What is the primary target variable that you want to predict? 

Answer: 

2.  Use Seaborn (`sns`) to explore the numeric data graphically. The primary goal is to understand if there any variables that seem likely to be helpful in building a model that can predict heart disease. 

In [ ]:
sns.pairplot(heart, vars = ['Age','RestBP','Chol','MaxHR','Oldpeak'], hue = "AHD")

Here I make an example of a count plot for a categorical variable.  You should look through all the categorical variables to answer the next question. 

In [ ]:
sns.countplot(heart,x = 'ExAng', hue = 'AHD')
#to run a different categorical variable change X 

What is a reasonable guess at which variables are going to help you make this prediction?  

Answer:

### 3. As the analysis plan is to use a Logistic Regression model, all variables will need to be converted to numeric representations. One way to do this is `pd.get_dummies` which I give an example below. The other way to do it is to use .map method and define an encoder. 

### I show both approaches below.  Typically, if there is an obvious hierarchical relationship, I might use my own encoder to express that.  If there isnt, get_dummies is more sensible then a hierarchical approach

### After you encode the heart data, feel free to delete my example code.  

In [ ]:
## I made a small example here, of how to use pd.get_dummies. Yes, you could do these steps manually. But 
## this is the slick way to do it. 
categories = [('Type',['Cat','Dog','Parrot','Whale']),
              ('Color',['Brown','Black','Mixed'])]
X = pd.DataFrame(columns=['Type','Color','Age'],
                       data=[['Cat','Brown',4.2],['Dog','Brown',3.2],['Parrot','Mixed',21]])

# This is quick and easy but can blow up your number of variables 
X_dummy = pd.get_dummies(X)
display(X, X_dummy)  #This is similar to head, but shows everything.  

#This is not so easy, and requires you think, but can be useful
encoder = {'Cat':1,'Dog':2,'Parrot':3,'Whale':4}
X_encoder = X['Type'].map(encoder)
X['Type_enc'] = X_encoder
display(X)

In our Heart DataFrame, Thal, AHD, and ChestPain will have to be encoded.  

In [ ]:
#Lets use get_dummies to do it. 

Heart_enc = pd.get_dummies(heart)
Heart_enc.info()

In [ ]:
Heart_enc.head()

In [ ]:
#Let's clean up the data frame by dropping the redundant ADHD_No and
Heart_enc = Heart_enc.drop(columns = ['Unnamed: 0','AHD_No'])
Heart_enc = Heart_enc.dropna()

In [ ]:
Heart_enc.head()

### 4.  First split the data into training and test data (using `train_test_split`). I recommend 25% test set data.   You can simply put the whole data frame in, and get two new data frames    

In [ ]:
training_Heart, test_Heart = train_test_split(Heart_enc,test_size = 0.25, random_state = 11 )

### 5. Fit a logistic regression model on the training data and assess model fit. Your assessment should include accuracy and a confusion matrix.  How well you do on the training data is not that informative.  But if you fail at the training data, you can never hope to predict the test data    

In [ ]:
cols = training_Heart.columns
print(cols)
logreg = LogisticRegression(random_state=16,max_iter = 5000)

# fit the model with data
logreg.fit(training_Heart[cols[0:18]], training_Heart['AHD_Yes'])
#first 18 columns of the data frame should contain predictors, the last column contains the target prediction 

In [ ]:
#accuracy can be obtained from score


In [ ]:
#lets get and plot a confusion matrix 

### 6. Test the logistic regression model on the testing data. Provide accuracy, confusion matrix, classification report, and ROC curve.  In particular with AUC value estimated from the ROC curve will tell us about the predictive power of the model, much like an $R^2$

7. Write a short paragraph describing this data, and what you see in it, and explaining how your logistic regression model performed in predicting heart disease from the variables.  What 3 variables do you think had the strongest influence on your ability to make these predictions? Why?  

### Geometric Interpretation of Logistic Regression Coefficients.

* Earlier we saw the general form of simple logistic regression, meaning when there
is just one predictor used in the model. 
* The model statement:
$$  \ln\left( \frac{P(Y = 1 \mid x)}{1 - P(Y = 1 \mid x)} \right) = \beta_0 + \beta_1 x $$

* in this case we were able to interpret the point where the log-odds is 0 (equal probability of the two classes)
and realize the decision boundary is 

$$ x = \frac{-\beta_0}{\beta_1} $$

* Multiple logistic regression is a generalization to multiple predictors. We can define a multiple logistic regression model to predict 𝑃(𝑌 = 1) as such:

$$ \ln \left( \frac{P(Y = 1 \mid X)}{1 - P(Y = 1 \mid X)} \right)
= \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_kx_k$$

where $X = \{x_1,x_2,x_3,x_4,...x_k \}$

* With two predictors, if we were to solve for the boundary, we would get an equation of a line 

$$ 0 = \beta_0+\beta_1x_1+\beta_2x_2 $$

* With 3 predictors its a plane, etc. etc.

* Another way of interpreting the log-odds form is that 



### Probability intepretation of Logistic Regression 

* In the case of one variable, there is also an easy probability interpretation of the logistic regression coefficients.  

$$  \ln\left( \frac{P(Y = 1 \mid x)}{1 - P(Y = 1 \mid x)} \right) = \beta_0 + \beta_1 x $$

* A one unit change in x is associated with an $e^{\beta_1}$ change in the odds that $𝑌 = 1$ .

* In the case of multiple regression, the same thing is true 

$$ \ln \left( \frac{P(Y = 1 \mid X)}{1 - P(Y = 1 \mid X)} \right)
= \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_kx_k$$

* This would suggest that we should interpret the size of $\beta_k$ as a meaure of the the importance of $x_k$ in the logistic regression model 

* However, the problem is each of the variables $x_k$ is in different units, and thus each of the $\beta_k$ also has different units.  

In [ ]:
print("Means")
print(np.mean(Heart_enc[cols[0:18]], axis = 0))


In [ ]:
print("Standard Deviations")
print(np.std(Heart_enc[cols[0:18]], axis = 0))

* As we can see, the values are on very different scales, because they are in different units. 
* log-odds does not have any units. 
* For example Age is in units of Years, while RestBP is in units mmHG, and ChestPain_typical has no units. 
* then $\beta_Age$ is in units of 1/Years while $\beta_RestBP$ is in units 1/mmHG 
* Thus, the $\beta$ values cannot be directly compared. 

###  Standardized variables. 

* One of the ways we can address this is to use standardized variables. Colloquially, someone might say z-score your variables. 

* Given a sample of variable $x$ with sample mean $\mu and sample standard deviation $\sigma$, we can standardize the variable as 

$$ z = \frac{x-\mu}{\sigma} $$

* The new variable z will have a mean of zero and a standard deviation of 1.  We estimate $\mu$ from the sample mean and $\sigma$ from the sample standard deviation.  

* `sklearn` has a neat method `StandardScaler` to do this. When we use `StandardScaler` we use the estimate of $\mu$ and $\sigma$ from the **training data only**


In [ ]:
ss = StandardScaler()  #Sadly StandardScaler does not yet play nice with pandas. 
X_train = training_Heart[cols[0:18]].values
Z_train = ss.fit_transform(X_train) 
X_test = test_Heart[cols[0:18]].values
Z_test = ss.transform(X_test) # notice that the test data is not being fit

* Extract the Y values to np array 

In [ ]:
Y_train = training_Heart['AHD_Yes'].values
Y_test = test_Heart['AHD_Yes'].values

8.  Make a new logistic regression model using the Standardized variables X_train, Y_train and test on X_test,Y_test.  Confirm that the model performance looks the same as the original variables bu examining the Classification Report and ROC curve.  They will not be exactly the same, but very close.  

If your model is called `logregz` then you can extract the $\beta$ coefficients in the variable `logregz.coef_`
If you want the interacept $\beta_0$ is found in `logregz.intercept_` 

These $beta_k$ capture the changes in odds for standardized variables.  This means the odds increases by $e^{\beta_k}$ when variable $k$ increases by one standard deviation $\sigma_k$

Now you can compare the $beta_k$ values directly and the largest one corresponds to the variable with the highest impact on prediction  

9.  Revise your paragraph summarizing the results of this model here, but now include a discussion of which 3 variables were the most important to the prediction.  